In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('Classify Readable Book')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters
auwpp_threshold = 0.3
wps_threshold = 67 / 60

In [5]:
from gorani.spark import read_data_all
import pyspark.sql.functions as F

time_df = read_data_all(spark, 'book_estimated_time')\
        .select('user_id', 'book_id', (F.col('estimated_time') / F.col('total_words')).alias('wps'))
time_df.show()

auwpp_df = read_data_all(spark, 'user_auwpps')
auwpp_df.show()

+-------+-------+--------------------+
|user_id|book_id|                 wps|
+-------+-------+--------------------+
|      1|      1|0.006869354610488...|
|      1|      2|0.007883862960282604|
|      1|      3|0.008877511374808172|
|      1|      4| 0.00824404921373138|
|      1|      5|0.004454638506915118|
|      1|      6|0.007305833782678706|
+-------+-------+--------------------+

+-------+-------+----------+
|user_id|book_id|     auwpp|
+-------+-------+----------+
|      1|      1| 0.9992443|
|      1|      2|0.99113923|
|      1|      3|0.99249524|
|      1|      4| 0.9906931|
|      1|      5| 0.9941841|
|      1|      6| 0.9980638|
+-------+-------+----------+



In [12]:
from gorani.spark import write_data
result_df = time_df.join(auwpp_df.alias('f'), (time_df['user_id'] == auwpp_df['user_id']) & (time_df['book_id'] == auwpp_df['book_id']), 'inner')\
        .drop(F.col('f.user_id'))\
        .drop(F.col('f.book_id'))\
        .where((F.col('auwpp') <= auwpp_threshold) & (F.col('wps') <= wps_threshold))\
        .drop(F.col('wps'))\
        .drop(F.col('auwpp'))

write_data('readable_books', result_df)

print('success')

success
